In [1]:
import pandas as pd
import numpy as np
import pybaseball as pb

In [5]:
df_2023 = pd.read_csv("C:/Users/wampl/sxwOBA/statcast_data/2023.csv")
sample_df = df_2023.sample(n=10000)
sample_df.to_csv('C:/Users/wampl/sxwOBA/statcast_data/sample_2023.csv', index=False)

In [2]:
current = pd.read_csv("statcast_data/current.csv")

In [3]:
current.loc[current['target'] == 7].head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,of_fielding_alignment,batter_team,pitcher_team,stand_L,stand_R,batter_name,rf_xwoba,sxwOBA,sxwoba_prob_model,pulled_barrel
33,33,SL,2023-06-26,91.3,-1.90,5.95,"Baumann, Mike",663886,657508,strikeout,...,0.0,CIN,BAL,0,1,tyler stephenson,0.0,0.0,NaN,0
80,80,CH,2023-06-26,83.9,-0.46,6.71,"Baker, Bryan",680574,641329,strikeout,...,0.0,CIN,BAL,0,1,matt mclain,0.0,0.0,NaN,0
84,84,FF,2023-06-26,97.9,-0.54,6.61,"Baker, Bryan",621028,641329,strikeout,...,0.0,CIN,BAL,0,1,kevin newman,0.0,0.0,NaN,0
177,177,FS,2023-06-26,81.0,-1.45,5.99,"Cruz, Fernando",669720,518585,strikeout,...,0.0,BAL,CIN,0,1,austin hays,0.0,0.0,NaN,0
183,183,FS,2023-06-26,80.8,-1.19,6.09,"Cruz, Fernando",622761,518585,strikeout,...,0.0,BAL,CIN,0,1,jorge mateo,0.0,0.0,NaN,0


In [4]:
df = pd.read_csv("statcast_data/2023.csv")

In [5]:
df.head().reset_index(drop=True)

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,0,FF,2023-06-26,96.6,1.30,5.46,"Pérez, Cionel",663697,672335,field_out,...,10,3,10,3,10,Standard,Standard,143.0,0.000,-0.470
1,1,SL,2023-06-26,84.0,1.13,5.41,"Pérez, Cionel",680574,672335,hit_by_pitch,...,10,3,10,3,10,Strategic,Standard,337.0,0.000,0.268
2,2,FF,2023-06-26,98.1,1.02,5.43,"Pérez, Cionel",680574,672335,NaN,...,10,3,10,3,10,Standard,Standard,135.0,0.000,-0.036
3,3,FF,2023-06-26,97.2,1.12,5.42,"Pérez, Cionel",621028,672335,field_out,...,10,3,10,3,10,Strategic,Standard,146.0,0.001,-0.305
4,4,FF,2023-06-26,98.2,1.17,5.40,"Pérez, Cionel",592200,672335,walk,...,10,3,10,3,10,Standard,Standard,138.0,-0.001,0.227


In [6]:
df.head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,0,FF,2023-06-26,96.6,1.30,5.46,"Pérez, Cionel",663697,672335,field_out,...,10,3,10,3,10,Standard,Standard,143.0,0.000,-0.470
1,1,SL,2023-06-26,84.0,1.13,5.41,"Pérez, Cionel",680574,672335,hit_by_pitch,...,10,3,10,3,10,Strategic,Standard,337.0,0.000,0.268
2,2,FF,2023-06-26,98.1,1.02,5.43,"Pérez, Cionel",680574,672335,NaN,...,10,3,10,3,10,Standard,Standard,135.0,0.000,-0.036
3,3,FF,2023-06-26,97.2,1.12,5.42,"Pérez, Cionel",621028,672335,field_out,...,10,3,10,3,10,Strategic,Standard,146.0,0.001,-0.305
4,4,FF,2023-06-26,98.2,1.17,5.40,"Pérez, Cionel",592200,672335,walk,...,10,3,10,3,10,Standard,Standard,138.0,-0.001,0.227


In [7]:
sprint_speed = pb.statcast_sprint_speed(2023, min_opp=5)
sprint_speed.head()

,last_name,first_name,player_id,team_id,team,position,age,competitive_runs,bolts,hp_to_1b,sprint_speed
0,De La Cruz,Elly,682829,113,CIN,3B,21,49,30.0,4.15,30.8
1,Witt Jr.,Bobby,677951,118,KC,SS,23,121,66.0,4.15,30.4
2,Thompson,Bubba,669352,140,TEX,LF,25,24,19.0,4.22,30.4
3,Turner,Trea,607208,143,PHI,SS,30,117,49.0,4.13,30.3
4,Carroll,Corbin,682998,109,AZ,LF,22,131,58.0,4.08,30.1


In [4]:
import pandas as pd
import os

# read in the dataframes
base_dir = "C:/Users/wampl/sxwOBA"
leaders_2021 = pd.read_csv(os.path.join(base_dir, "leaders", "spray_xwoba_2021.csv"))
leaders_2022 = pd.read_csv(os.path.join(base_dir, "leaders", "spray_xwoba_2022.csv"))
leaders_2023 = pd.read_csv(os.path.join(base_dir, "leaders", "spray_xwoba_2023.csv"))

# merge the dataframes on the 'batter' column
merged_leaders = pd.merge(leaders_2021, leaders_2022, on='batter', suffixes=('_2021', '_2022'))
merged_leaders = pd.merge(merged_leaders, leaders_2023, on='batter', suffixes=('', '_2023'))

In [5]:
merged_leaders.loc[(merged_leaders['PA_2021']>300) & (merged_leaders['PA_2022']>300)].corr()[['wOBA', 'wOBA_2022']]


C:\Users\wampl\AppData\Local\Temp\ipykernel_20336\4171482807.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_leaders.loc[(merged_leaders['PA_2021']>300) & (merged_leaders['PA_2022']>300)].corr()[['wOBA', 'wOBA_2022']]


,wOBA,wOBA_2022
batter,0.108966,0.063766
PA_2021,0.167567,0.307540
wOBA_2021,0.306904,0.464917
xwOBA_2021,0.393755,0.521283
sxwOBA_2021,0.404682,0.527477
diff_2021,-0.021274,-0.054874
diff%_2021,-0.032209,-0.092631
BB%_2021,0.269205,0.177710
K%_2021,-0.123615,-0.109015
HR_2021,0.271092,0.419307


In [10]:
df = merged_leaders[['wOBA_2021', 'wOBA_2022', 'wOBA', 'xwOBA_2021', 'xwOBA_2022', 'xwOBA', 'sxwOBA_2021', 'sxwOBA_2022', 'sxwOBA']]

In [11]:
import plotly.express as px
merged_leaders['weighted_woba_2022_2023'] = ((merged_leaders['wOBA_2022']*merged_leaders['PA_2022'])+(merged_leaders['wOBA']*merged_leaders['PA'])) / (merged_leaders['PA_2022'] + merged_leaders['PA'])
merged_leaders = merged_leaders.round(3)

In [12]:
px.scatter(data_frame=merged_leaders.loc[merged_leaders['PA_2021'] > 200], x='xwOBA_2021', y='sxwOBA_2021', color='weighted_woba_2022_2023', hover_data=['Name', 'PA_2021'])

In [13]:
px.scatter(data_frame=merged_leaders.loc[merged_leaders['PA_2021'] > 200], x='xwOBA_2021', y='weighted_woba_2022_2023', trendline='ols',hover_data=['Name', 'PA_2021'])

In [14]:
px.scatter(data_frame=merged_leaders.loc[merged_leaders['PA_2021'] > 200], x='sxwOBA_2021', y='weighted_woba_2022_2023', trendline='ols',hover_data=['Name', 'PA_2021'])

In [15]:
merged_leaders.loc[(merged_leaders['PA_2021']>300) & (merged_leaders['PA_2022']>300)].corr().round(2)[['wOBA', 'wOBA_2022', 'weighted_woba_2022_2023']]

C:\Users\wampl\AppData\Local\Temp\ipykernel_18912\2764258261.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,wOBA,wOBA_2022,weighted_woba_2022_2023
batter,0.12,0.06,0.11
PA_2021,0.17,0.31,0.31
wOBA_2021,0.32,0.46,0.50
xwOBA_2021,0.41,0.52,0.57
sxwOBA_2021,0.41,0.53,0.57
diff_2021,-0.05,-0.06,-0.07
diff%_2021,-0.06,-0.10,-0.11
BB%_2021,0.25,0.18,0.25
K%_2021,-0.11,-0.11,-0.10
HR_2021,0.29,0.42,0.44


In [16]:
merged_leaders.corr()[['xwOBA_2021', 'xwOBA_2022', 'xwOBA', 'sxwOBA_2021', 'sxwOBA_2022', 'sxwOBA', 'weighted_woba_2022_2023']]

C:\Users\wampl\AppData\Local\Temp\ipykernel_18912\3546130828.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,xwOBA_2021,xwOBA_2022,xwOBA,sxwOBA_2021,sxwOBA_2022,sxwOBA,weighted_woba_2022_2023
batter,-0.192785,-0.112765,-0.003013,-0.156760,-0.058444,-0.024414,-0.026492
PA_2021,0.475474,0.399038,0.285704,0.472975,0.364846,0.310559,0.373656
wOBA_2021,0.806209,0.387855,0.240683,0.882557,0.350721,0.215739,0.372852
xwOBA_2021,1.000000,0.541533,0.397005,0.947180,0.466332,0.369322,0.522129
sxwOBA_2021,0.947180,0.506733,0.364515,1.000000,0.461454,0.335578,0.504773
diff_2021,-0.082221,-0.063781,-0.068443,0.241739,0.023046,-0.074609,-0.011163
diff%_2021,-0.191472,-0.089049,-0.105689,0.126569,0.004333,-0.117168,-0.063794
BB%_2021,0.442049,0.145540,0.160459,0.479318,0.110670,0.155788,0.198681
K%_2021,-0.282067,-0.163983,-0.111277,-0.331625,-0.206915,-0.133783,-0.203825
HR_2021,0.616922,0.464997,0.342418,0.586518,0.397086,0.334357,0.430863
